<a href="https://colab.research.google.com/github/steffenmodest/Python_Examples/blob/Protocol/M_Net_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import os.path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

rs = 23  # fester Random Seed

Load Data

In [67]:
from tensorflow.keras.datasets import fashion_mnist, mnist

# Fashion MNIST Daten laden, wir wollen nur die Trainingsdaten und verwerfen die Testdaten
# (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 28, 28)
(10000, 28, 28)
(60000,)
(10000,)


Own Funs

In [68]:
def ifelse(a):
  if a == 0:
    return 1
  else:
    return a


def rms_norm(v_x):
  n_rms = np.sqrt(np.sum(v_x ** 2)/(len(v_x) - 1))
  return v_x/n_rms


def prot_row(d_results):
    """
    make a new row in protokoll

    Parameters
    ----------
    outcome : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    """
    
    prot_file = 'M-Net_Protocol.xlsx'
    if os.path.isfile(prot_file):
        df_prot = pd.read_excel (prot_file)
        df_prot = df_prot.append(d_results, 
                          ignore_index=True)
    else:
        df_prot = pd.DataFrame.from_dict(d_results)
        # df_prot = pd.DataFrame(columns=('TIMESTAMP', 
        #                                 'NUMBER_OF_HIDDEN', 
        #                                 'ACCURACY_SCORE', 
        #                                 'MAE_1ST_FL',
        #                                 'MSE_1ST_FL',
        #                                 'R2_1ST_FL',
        #                                 'MAE_1ST_BA',
        #                                 'MSE_1ST_BA',
        #                                 'R2_1ST_BA',
        #                                 'MAE_2ND_FL',
        #                                 'MSE_2ND_FL',
        #                                 'R2_2ND_FL',
        #                                 'MAE_2ND_BA',
        #                                 'MSE_2ND_BA',
        #                                 'R2_2ND_BA',
        #                                 'MSG'))
        
    # s_outcome = outcome.partition('<msg>')[2]
    # s_outcome = s_outcome.partition('</msg>')[0]
    # s_outcome = s_outcome.partition('Set ')[2]
    # s_set = s_outcome.partition(' ')[0]
    # s_msg = s_outcome.partition(' ')[2]
    
    # s_outcome = s_outcome.strip(',()')
    # l_outcome = s_outcome.split(',')
    # df_prot = df_prot.append({'TIMESTAMP':datetime.now(),
    #                           'RESULT': result,
    #                           'SAS_SET':int(s_set),
    #                           'MSG':s_msg,
    #                           'COUNT':n_len}, ignore_index=True)
    
    # df_prot = df_prot.append(d_results, 
    #                          ignore_index=True)
    df_prot.to_excel(prot_file, index=False)
        
    return None

Params and Results

In [69]:
params = {'N_HIDDEN': 47}
results = {'NUMBER_OF_HIDDEN': params['N_HIDDEN'],
           'TIMESTAMP': datetime.now(),
          'ACCURACY_SCORE': 0, 
          'MAE_1ST_FL': 0,
          'MSE_1ST_FL': 0,
          'R2_1ST_FL': 0,
          'MAE_1ST_BA': 0,
          'MSE_1ST_BA': 0,
          'R2_1ST_BA': 0,
          'MAE_2ND_FL': 0,
          'MSE_2ND_FL': 0,
          'R2_2ND_FL': 0,
          'MAE_2ND_BA': 0,
          'MSE_2ND_BA': 0,
          'R2_2ND_BA': 0,
          'MSG':'M-Net on python'}

In [70]:
results

{'ACCURACY_SCORE': 0,
 'MAE_1ST_BA': 0,
 'MAE_1ST_FL': 0,
 'MAE_2ND_BA': 0,
 'MAE_2ND_FL': 0,
 'MSE_1ST_BA': 0,
 'MSE_1ST_FL': 0,
 'MSE_2ND_BA': 0,
 'MSE_2ND_FL': 0,
 'MSG': 'M-Net on python',
 'NUMBER_OF_HIDDEN': 47,
 'R2_1ST_BA': 0,
 'R2_1ST_FL': 0,
 'R2_2ND_BA': 0,
 'R2_2ND_FL': 0,
 'TIMESTAMP': datetime.datetime(2020, 12, 25, 22, 43, 17, 75503)}

Preprocess data

NEW Norm

In [71]:
X = X_train.reshape(len(X_train), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [72]:
X

array([[0.01148061, 0.01148061, 0.01148061, ..., 0.01148061, 0.01148061,
        0.01148061],
       [0.01076957, 0.01076957, 0.01076957, ..., 0.01076957, 0.01076957,
        0.01076957],
       [0.01425251, 0.01425251, 0.01425251, ..., 0.01425251, 0.01425251,
        0.01425251],
       ...,
       [0.01299969, 0.01299969, 0.01299969, ..., 0.01299969, 0.01299969,
        0.01299969],
       [0.01400668, 0.01400668, 0.01400668, ..., 0.01400668, 0.01400668,
        0.01400668],
       [0.01375564, 0.01375564, 0.01375564, ..., 0.01375564, 0.01375564,
        0.01375564]])

In [73]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, FunctionTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X = X_train.reshape(len(X_train), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X = preproccessing.fit_transform(X)
# X = np.array(X)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_train.shape, np.mean(X_train), np.std(X_train)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

Originaldaten:
Shape: (60000, 28, 28), Mean: 33.318421, STD: 78.567490
Vorbereitete Daten:
Shape: (60000, 784), Mean: 0.396041, STD: 0.917538


Create Hidden Activations

In [74]:
H = np.random.rand(60000, params['N_HIDDEN'])
print(H.shape)
print(H)

(60000, 47)
[[0.57161354 0.74193162 0.14204891 ... 0.80448902 0.86801864 0.37255938]
 [0.94675784 0.30667236 0.24833681 ... 0.57528916 0.23637273 0.88346985]
 [0.06279473 0.62744582 0.93754911 ... 0.69908185 0.36464326 0.30926249]
 ...
 [0.14566916 0.16945184 0.5925007  ... 0.27472119 0.32881081 0.90647449]
 [0.04375342 0.70413885 0.85130746 ... 0.53935067 0.07641706 0.27555326]
 [0.17952143 0.08185114 0.32274855 ... 0.90211449 0.41367784 0.94811836]]


1st Forward Learn

In [75]:
import numpy as np
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(H, X)
print(reg.score(H, X))
X_pred = reg.predict(H)

results['MAE_1ST_FL'] = round(mean_absolute_error(X_pred, X), 3)
results['MSE_1ST_FL'] = round(mean_squared_error(X_pred, X), 3)
results['R2_1ST_FL'] = round(r2_score(X_pred, X, multioutput="variance_weighted"), 3)

print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.0008177688469181434
MAE 0.4375412375741973
MSE 0.5934203469736058
R2 -1268.9935048311568
R2 vw -1220.839441449769
R2 ua -1268.9935048311568
(784, 47)
(784,)


1st Back Activations

In [76]:
# # X = X_train.reshape(len(X_train), -1).astype('int')
# W = pd.DataFrame(W)
# # X = X.applymap(ifelse)
# W = W.apply(rms_norm)
# W = np.array(X)

In [77]:
XT = X.transpose()
print(f'shape of X_train {X.shape} and X_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

shape of X_train (60000, 784) and X_train transposed (784, 60000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.5779704326531672
MAE 0.43507835910660775
MSE 0.3539219070940361
R2 0.2407129638317081
R2 vw 0.2698076865117301
R2 ua 0.2407129638317081
(60000, 47)
(60000,)


Hidden Norm

In [78]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

2nd Forward learn - result are Coherent Weights

In [79]:
reg = LinearRegression().fit(H_new, X)
print(reg.score(H_new, X))
X_pred = reg.predict(H_new)
print(f'MAE {mean_absolute_error(X_pred, X)}')
print(f'MSE {mean_squared_error(X_pred, X)}')
print(f'R2 {r2_score(X_pred, X)}')
print(f'R2 vw {r2_score(X_pred, X, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(X_pred, X, multioutput="uniform_average")}')

W = reg.coef_
print(W.shape)
WI = reg.intercept_
print(WI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7415405530922463
MAE 0.21213190958005634
MSE 0.1535006226898237
R2 -12.694696294736747
R2 vw 0.6514560858068112
R2 ua -12.694696294736747
(784, 47)
(784,)


2nd Back Activation - final representation of X_train in Hidden Activations

In [80]:
reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_new = reg.coef_
print(H_new.shape)
HI = reg.intercept_
print(HI.shape)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7847848664496424
MAE 0.2949563578603455
MSE 0.1804834456990609
R2 vw 0.7257654387195458
R2 ua 0.7153783760891832
(60000, 47)
(60000,)


In [81]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_new = pd.DataFrame(H_new)
# H_new = H_new.applymap(ifelse)
H_new = H_new.apply(rms_norm, axis=1)
H_new = np.array(H_new)

KNN test with Hidden Activations

In [82]:
X = X_test.reshape(len(X_test), -1).astype('int')
X = pd.DataFrame(X)
X = X.applymap(ifelse)
X = X.apply(rms_norm, axis=1)
X = np.array(X)

In [83]:
# X = [[0], [1], [2], [3]]
# y = [0, 0, 1, 1]

# Plätte 2D Bild in einen Vektor:
# Reshape behält die Anzahl Element in der ersten Dimension (len(X_orig) -> #Bilder)
# die restlichen Zellen des Arrays (-1 -> Pixel) bekommen alle ihre eigene Dimension
# X_p_test = X_test.reshape(len(X_test), -1).astype('float64')

# # Dimensionen um den Mittelpunkt zentrieren
# preproccessing = StandardScaler()
# X_p_test = preproccessing.fit_transform(X_p_test)

print ("Originaldaten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X_test.shape, np.mean(X_test), np.std(X_test)))

print ("Vorbereitete Daten:")
print("Shape: {}, Mean: {:f}, STD: {:f}".format(X.shape, np.mean(X), np.std(X)))

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, n_jobs=-1, weights='distance')
neigh.fit(H_new, y_train)

# print(neigh.predict([[1.1]]))

# print(neigh.predict_proba([[0.9]]))

Originaldaten:
Shape: (10000, 28, 28), Mean: 33.791224, STD: 79.172463
Vorbereitete Daten:
Shape: (10000, 784), Mean: 0.398067, STD: 0.916661


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='distance')

Back Activation of Test Data

In [84]:
XT = X.transpose()
print(f'shape of X_p_train {X.shape} and X_p_train transposed {XT.shape}')

reg = LinearRegression().fit(W, XT)
print(reg.score(W, XT))
XT_pred = reg.predict(W)
print(f'MAE {mean_absolute_error(XT_pred, XT)}')
print(f'MSE {mean_squared_error(XT_pred, XT)}')
print(f'R2 {r2_score(XT_pred, XT)}')
print(f'R2 vw {r2_score(XT_pred, XT, multioutput="variance_weighted")}')
print(f'R2 ua {r2_score(XT_pred, XT, multioutput="uniform_average")}')

H_test = reg.coef_
print(H_test.shape)
HI_test = reg.intercept_
print(HI_test.shape)

shape of X_p_train (10000, 784) and X_p_train transposed (784, 10000)


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.7900643108438795
MAE 0.2923868072633915
MSE 0.1757155981323575
R2 0.7253290369879228
R2 vw 0.7342802525380691
R2 ua 0.7253290369879228
(10000, 47)
(10000,)


In [85]:
# X = H_new_train.reshape(len(H_new_train), -1).astype('int')
H_test = pd.DataFrame(H_test)
# H_new = H_new.applymap(ifelse)
H_test = H_test.apply(rms_norm, axis=1)
H_test = np.array(H_test)

In [86]:
# Predicting the Test set results
y_pred = neigh.predict(H_test)

In [87]:
# y_prob = neigh.predict_proba(H_test)

In [88]:
# print(y_prob[0])

In [89]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 975    1    1    0    0    0    2    1    0    0]
 [   0 1129    2    0    0    1    3    0    0    0]
 [   8    1 1003    2    1    0    1   10    6    0]
 [   3    0    3  974    1   11    0    6    8    4]
 [   3    7    0    0  937    0    7    0    1   27]
 [   3    0    1    9    2  857    9    1    7    3]
 [   8    4    0    0    1    1  943    0    1    0]
 [   0   11    5    1    3    0    0  995    1   12]
 [   5    0    3    5    3    5    4    3  942    4]
 [   5    5    4    6    7    3    1    6    4  968]]


In [90]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.9723
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.97      0.98      1032
           3       0.98      0.96      0.97      1010
           4       0.98      0.95      0.97       982
           5       0.98      0.96      0.97       892
           6       0.97      0.98      0.98       958
           7       0.97      0.97      0.97      1028
           8       0.97      0.97      0.97       974
           9       0.95      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000



In [91]:
prot_row(results)